reference: https://medium.com/@daniel820710/%E5%88%A9%E7%94%A8keras%E5%BB%BA%E6%A7%8Blstm%E6%A8%A1%E5%9E%8B-%E4%BB%A5stock-prediction-%E7%82%BA%E4%BE%8B-1-67456e0a0b

# Import

In [ ]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, LSTM, TimeDistributed, RepeatVector
from keras.layers.normalization import BatchNormalization
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ModelCheckpoint
import matplotlib.pyplot as plt
%matplotlib inline

# Define Functions

In [ ]:
# Read Data
# SPY dataset: Yahoo SPDR S&P 500 ETF (SPY):http://goo.gl/QXikaf
def readTrain():
  train = pd.read_csv("SPY.csv")
  return train

# Augment Features
def augFeatures(train):
  train["Date"] = pd.to_datetime(train["Date"])
  train["year"] = train["Date"].dt.year
  train["month"] = train["Date"].dt.month
  train["date"] = train["Date"].dt.day
  train["day"] = train["Date"].dt.dayofweek
  return train

# Normalization
def normalize(train):
  train = train.drop(["Date"], axis=1)
  train_norm = train.apply(lambda x: (x - np.mean(x)) / (np.max(x) - np.min(x)))
  return train_norm

# Build Train
def buildTrain(train, pastDay=30, futureDay=5):
  X_train, Y_train = [], []
  for i in range(train.shape[0]-futureDay-pastDay):
    X_train.append(np.array(train.iloc[i:i+pastDay]))
    Y_train.append(np.array(train.iloc[i+pastDay:i+pastDay+futureDay]["Adj Close"]))
  return np.array(X_train), np.array(Y_train)

# Shuffle Data
def shuffle(X,Y):
  np.random.seed(10)
  randomList = np.arange(X.shape[0])
  np.random.shuffle(randomList)
  return X[randomList], Y[randomList]

# Train/Valid Split
def splitData(X,Y,rate):
  X_train = X[int(X.shape[0]*rate):]
  Y_train = Y[int(Y.shape[0]*rate):]
  X_val = X[:int(X.shape[0]*rate)]
  Y_val = Y[:int(Y.shape[0]*rate)]
  return X_train, Y_train, X_val, Y_val

# Dataset Preparing

In [ ]:
# read SPY.csv
train = readTrain()
train.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,1993-01-29,43.96875,43.96875,43.75000,43.93750,25.968958,1003200
1,1993-02-01,43.96875,44.25000,43.96875,44.25000,26.153660,480500
2,1993-02-02,44.21875,44.37500,44.12500,44.34375,26.209057,201300
3,1993-02-03,44.40625,44.84375,44.37500,44.81250,26.486113,529400
4,1993-02-04,44.96875,45.09375,44.46875,45.00000,26.596937,531500


In [ ]:
# Augment the features (year, month, date, day)
train_Aug = augFeatures(train)
train_Aug.head()

,Date,Open,High,Low,Close,Adj Close,Volume,year,month,date,day
0,1993-01-29,43.96875,43.96875,43.75000,43.93750,25.968958,1003200,1993,1,29,4
1,1993-02-01,43.96875,44.25000,43.96875,44.25000,26.153660,480500,1993,2,1,0
2,1993-02-02,44.21875,44.37500,44.12500,44.34375,26.209057,201300,1993,2,2,1
3,1993-02-03,44.40625,44.84375,44.37500,44.81250,26.486113,529400,1993,2,3,2
4,1993-02-04,44.96875,45.09375,44.46875,45.00000,26.596937,531500,1993,2,4,3


In [ ]:
# Normalization
train_norm = normalize(train_Aug)
train_norm.head()

,Open,High,Low,Close,Adj Close,Volume,year,month,date,day
0,-0.342052,-0.344423,-0.340762,-0.341338,-0.301050,-0.095759,-0.486789,-0.501146,0.442720,0.494918
1,-0.342052,-0.343266,-0.339857,-0.340053,-0.300297,-0.096359,-0.486789,-0.410237,-0.490614,-0.505082
2,-0.341021,-0.342752,-0.339210,-0.339668,-0.300072,-0.096680,-0.486789,-0.410237,-0.457280,-0.255082
3,-0.340248,-0.340824,-0.338176,-0.337740,-0.298942,-0.096303,-0.486789,-0.410237,-0.423947,-0.005082
4,-0.337930,-0.339796,-0.337788,-0.336969,-0.298490,-0.096301,-0.486789,-0.410237,-0.390614,0.244918


In [ ]:
# build Data, use last 30 days to predict next 5 days
X_train, Y_train = buildTrain(train_norm, 30, 5)

# shuffle the data, and random seed is 10
X_train, Y_train = shuffle(X_train, Y_train)

# split training data and validation data
X_train, Y_train, X_val, Y_val = splitData(X_train, Y_train, 0.1)

In [ ]:
print("X_train: ", X_train.shape)
print("Y_train: ", Y_train.shape)
print("X_val: ", X_val.shape)
print("Y_val: ", Y_val.shape)

X_train:  (5680, 30, 10)
Y_train:  (5680, 5)
X_val:  (631, 30, 10)
Y_val:  (631, 5)


In [ ]:
train.head()

,Date,Open,High,Low,Close,Adj Close,Volume,year,month,date,day
0,1993-01-29,43.96875,43.96875,43.75000,43.93750,25.968958,1003200,1993,1,29,4
1,1993-02-01,43.96875,44.25000,43.96875,44.25000,26.153660,480500,1993,2,1,0
2,1993-02-02,44.21875,44.37500,44.12500,44.34375,26.209057,201300,1993,2,2,1
3,1993-02-03,44.40625,44.84375,44.37500,44.81250,26.486113,529400,1993,2,3,2
4,1993-02-04,44.96875,45.09375,44.46875,45.00000,26.596937,531500,1993,2,4,3


In [ ]:
train_Aug.head()

,Date,Open,High,Low,Close,Adj Close,Volume,year,month,date,day
0,1993-01-29,43.96875,43.96875,43.75000,43.93750,25.968958,1003200,1993,1,29,4
1,1993-02-01,43.96875,44.25000,43.96875,44.25000,26.153660,480500,1993,2,1,0
2,1993-02-02,44.21875,44.37500,44.12500,44.34375,26.209057,201300,1993,2,2,1
3,1993-02-03,44.40625,44.84375,44.37500,44.81250,26.486113,529400,1993,2,3,2
4,1993-02-04,44.96875,45.09375,44.46875,45.00000,26.596937,531500,1993,2,4,3


# Many-to-One Model

In [ ]:
def buildManyToOneModel(shape):
  model = Sequential()
  model.add(LSTM(10, input_length=shape[1], input_dim=shape[2]))
  # output shape: (1, 1)
  model.add(Dense(1))
  model.compile(loss="mse", optimizer="adam")
  model.summary()
  return model

In [ ]:
train = readTrain()
train_Aug = augFeatures(train)
train_norm = normalize(train_Aug)
# change the last day and next day 
X_train, Y_train = buildTrain(train_norm, 30, 1)
X_train, Y_train = shuffle(X_train, Y_train)
# because no return sequence, Y_train and Y_val shape must be 2 dimension
X_train, Y_train, X_val, Y_val = splitData(X_train, Y_train, 0.1)

model = buildManyToOneModel(X_train.shape)
callback = EarlyStopping(monitor="loss", patience=10, verbose=1, mode="auto")
model.fit(X_train, Y_train, epochs=1000, batch_size=128, validation_data=(X_val, Y_val), callbacks=[callback])


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 10)                840       
_________________________________________________________________
dense (Dense)                (None, 1)                 11        
Total params: 851
Trainable params: 851
Non-trainable params: 0
_________________________________________________________________
Epoch 1/1000
45/45 [==============================] - 8s 13ms/step - loss: 0.0175 - val_loss: 0.0020
Epoch 2/1000
45/45 [==============================] - 0s 5ms/step - loss: 0.0016 - val_loss: 7.8499e-04
Epoch 3/1000
45/45 [==============================] - 0s 4ms/step - loss: 7.0939e-04 - val_loss: 4.4657e-04
Epoch 4/1000
45/45 [==============================] - 0s 4ms/step - loss: 4.3224e-04 - val_loss: 3.0514e-04
Epoch 5/1000
45/45 [==============================] - 0s 4ms/step - loss: 3.0726e-04 - val_loss: 2.

# Many-to-Many Model (5-5)

In [ ]:
def buildManyToManyModel(shape):
  model = Sequential()
  model.add(LSTM(10, input_length=shape[1], input_dim=shape[2], return_sequences=True))
  # output shape: (5, 1)
  model.add(TimeDistributed(Dense(1)))
  model.compile(loss="mse", optimizer="adam")
  model.summary()
  return model

In [ ]:
train = readTrain()
train_Aug = augFeatures(train)
train_norm = normalize(train_Aug)
# change the last day and next day 
X_train, Y_train = buildTrain(train_norm, 5, 5)
X_train, Y_train = shuffle(X_train, Y_train)
X_train, Y_train, X_val, Y_val = splitData(X_train, Y_train, 0.1)

# from 2 dimmension to 3 dimension
Y_train = Y_train[:,:,np.newaxis]
Y_val = Y_val[:,:,np.newaxis]

model = buildManyToManyModel(X_train.shape)
callback = EarlyStopping(monitor="loss", patience=10, verbose=1, mode="auto")
model.fit(X_train, Y_train, epochs=1000, batch_size=128, validation_data=(X_val, Y_val), callbacks=[callback])

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 5, 10)             840       
_________________________________________________________________
time_distributed (TimeDistri (None, 5, 1)              11        
Total params: 851
Trainable params: 851
Non-trainable params: 0
_________________________________________________________________
Epoch 1/1000
45/45 [==============================] - 7s 11ms/step - loss: 0.0392 - val_loss: 0.0106
Epoch 2/1000
45/45 [==============================] - 0s 3ms/step - loss: 0.0098 - val_loss: 0.0073
Epoch 3/1000
45/45 [==============================] - 0s 3ms/step - loss: 0.0072 - val_loss: 0.0056
Epoch 4/1000
45/45 [==============================] - 0s 3ms/step - loss: 0.0056 - val_loss: 0.0044
Epoch 5/1000
45/45 [==============================] - 0s 3ms/step - loss: 0.0045 - val_loss: 0.0034
Epoch 6/1000
45/45 

# Many-to-Many(30-5)

In [ ]:
def buildManyToManyModel(shape):
  model = Sequential()  
  model.add(LSTM(32, input_shape=(shape[1], shape[2]), return_sequences=False))  
  model.add(RepeatVector(5)) 
  model.add(LSTM(32, return_sequences=True))  
  model.add(TimeDistributed(Dense(1))) 
  model.compile(loss="mse", optimizer="adam")
  model.summary()
  return model

In [ ]:
train = readTrain()
train_Aug = augFeatures(train)
train_norm = normalize(train_Aug)
# change the last day and next day 
X_train, Y_train = buildTrain(train_norm, 30, 5)
X_train, Y_train = shuffle(X_train, Y_train)
X_train, Y_train, X_val, Y_val = splitData(X_train, Y_train, 0.1)

# from 2 dimmension to 3 dimension
Y_train = Y_train[:,:,np.newaxis]
Y_val = Y_val[:,:,np.newaxis]

In [ ]:
X_train.shape

(5680, 30, 10)

In [ ]:
Y_train.shape

(5680, 5, 1)

In [ ]:
model = buildManyToManyModel(X_train.shape)
callback = EarlyStopping(monitor="loss", patience=10, verbose=1, mode="auto")
model.fit(X_train, Y_train, epochs=1000, batch_size=128, validation_data=(X_val, Y_val), callbacks=[callback])

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 32)                5504      
_________________________________________________________________
repeat_vector (RepeatVector) (None, 5, 32)             0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 5, 32)             8320      
_________________________________________________________________
time_distributed (TimeDistri (None, 5, 1)              33        
Total params: 13,857
Trainable params: 13,857
Non-trainable params: 0
_________________________________________________________________
Epoch 1/1000
45/45 [==============================] - 34s 24ms/step - loss: 0.0167 - val_loss: 0.0031
Epoch 2/1000
45/45 [==============================] - 0s 6ms/step - loss: 0.0022 - val_loss: 8.3221e-04
Epoch 3/1000
45/45 [========================

In [ ]:
from math import sqrt
from matplotlib import pyplot
from sklearn.metrics import accuracy_score, mean_squared_error

"""
pred_many = model.predict(X_val)
rmse = sqrt(mean_squared_error(Y_val, pred_many))
fig = pyplot.figure()
ax = fig.add_subplot(111)
ax.plot(Y_val, label="Real")
ax.legend(loc='upper left')
pyplot.plot(pred_many, label="Prediction")
pyplot.legend(loc='upper left')
pyplot.savefig(imgPath)
pyplot.clf()
"""
pred = model.predict(X_val)
pred

array([[[-0.29861587],
        [-0.29575658],
        [-0.29669905],
        [-0.2966503 ],
        [-0.2959519 ]],

       [[-0.06397185],
        [-0.06455555],
        [-0.06446442],
        [-0.06443556],
        [-0.06440888]],

       [[-0.04232759],
        [-0.04122655],
        [-0.04029436],
        [-0.03946568],
        [-0.03870175]],

       ...,

       [[-0.01302751],
        [-0.01178204],
        [-0.01201375],
        [-0.01241651],
        [-0.01276767]],

       [[-0.23128599],
        [-0.23170087],
        [-0.23077396],
        [-0.23003304],
        [-0.22925347]],

       [[-0.07692419],
        [-0.07753175],
        [-0.07773903],
        [-0.07807811],
        [-0.07831961]]], dtype=float32)

In [ ]:
Y_val

array([[[-0.30032349],
        [-0.30039908],
        [-0.30002092],
        [-0.30009651],
        [-0.29790265]],

       [[-0.06545521],
        [-0.06419736],
        [-0.05992023],
        [-0.06000415],
        [-0.06676909]],

       [[-0.0438063 ],
        [-0.0442977 ],
        [-0.04406844],
        [-0.04030028],
        [-0.03928453]],

       ...,

       [[-0.01402029],
        [-0.01716596],
        [-0.02595749],
        [-0.01725236],
        [-0.01302905]],

       [[-0.23173047],
        [-0.23111652],
        [-0.22960197],
        [-0.23025693],
        [-0.23062529]],

       [[-0.08532259],
        [-0.08270411],
        [-0.0892644 ],
        [-0.08687113],
        [-0.0883352 ]]])

In [ ]:
from numpy import ndarray
Y_val = Y_val.flatten()
pred = pred.flatten()
rmse = sqrt(mean_squared_error(Y_val, pred))

In [ ]:
rmse

0.009046073094185793

In [ ]:
from keras import regularizers
def buildManyToManyModel(shape):
  model = Sequential() 
  model.add(LSTM(32, input_shape=(shape[1], shape[2]), 
                    kernel_regularizer=regularizers.l2(0.01), 
                    recurrent_regularizer=regularizers.l2(0.01), 
                    bias_regularizer=regularizers.l2(0.01)))
  model.add(RepeatVector(5)) 
  model.add(Dense(32,
                    kernel_regularizer=regularizers.l2(0.01), 
                    bias_regularizer=regularizers.l2(0.01)))
  model.add(Dropout(0.02))
  model.add(TimeDistributed(Dense(1)))  
  model.compile(loss="mse", optimizer="adam")
  model.summary()
  return model

In [ ]:
model = buildManyToManyModel(X_train.shape)
callback = EarlyStopping(monitor="loss", patience=10, verbose=1, mode="auto")
model.fit(X_train, Y_train, epochs=1000, batch_size=128, validation_data=(X_val, Y_val), callbacks=[callback])

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_2 (LSTM)                (None, 32)                5504      
_________________________________________________________________
repeat_vector_2 (RepeatVecto (None, 5, 32)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 5, 32)             1056      
_________________________________________________________________
dropout (Dropout)            (None, 5, 32)             0         
_________________________________________________________________
time_distributed (TimeDistri (None, 5, 1)              33        
Total params: 6,593
Trainable params: 6,593
Non-trainable params: 0
_________________________________________________________________
Epoch 1/1000
45/45 [==============================] - 31s 21ms/step - loss: 1.0773 - val_loss: 0.7732
Epoch 2/1000
45/

In [ ]:
from math import sqrt
from matplotlib import pyplot
from sklearn.metrics import accuracy_score, mean_squared_error

"""
pred_many = model.predict(X_val)
rmse = sqrt(mean_squared_error(Y_val, pred_many))
fig = pyplot.figure()
ax = fig.add_subplot(111)
ax.plot(Y_val, label="Real")
ax.legend(loc='upper left')
pyplot.plot(pred_many, label="Prediction")
pyplot.legend(loc='upper left')
pyplot.savefig(imgPath)
pyplot.clf()
"""
pred = model.predict(X_val)
pred

array([[[-0.3092832 ],
        [-0.3092832 ],
        [-0.3092832 ],
        [-0.3092832 ],
        [-0.3092832 ]],

       [[-0.05640321],
        [-0.05640321],
        [-0.05640321],
        [-0.05640321],
        [-0.05640321]],

       [[-0.04012572],
        [-0.04012572],
        [-0.04012572],
        [-0.04012572],
        [-0.04012572]],

       ...,

       [[-0.00244225],
        [-0.00244225],
        [-0.00244225],
        [-0.00244225],
        [-0.00244225]],

       [[-0.23745273],
        [-0.23745273],
        [-0.23745273],
        [-0.23745273],
        [-0.23745273]],

       [[-0.07253215],
        [-0.07253215],
        [-0.07253215],
        [-0.07253215],
        [-0.07253215]]], dtype=float32)

In [ ]:
from numpy import ndarray
Y_val = Y_val.flatten()
pred = pred.flatten()
rmse = sqrt(mean_squared_error(Y_val, pred))
print(rmse)

0.012478853479115287
